#Feature Extraction

The aim of this jupyter notebook is to build upon the data file extraction made by Amanda. The main idea is to use pre
existing libraries susch as *scikit-learn*. The main goal is to learn the basics, and train a machine learning model
quickly and easily.

In [138]:
# First we will extract the data, and save it to a csv.
# LAMAO ERLEND. HVOR ER DU? hahahhahahaha
# Er hjemmme 😅 skal få H2O, aka autoML til å funke :
# Hva er H20? SMUDE! 
# You go !!
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [139]:
train_a = pd.read_parquet('A/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_train_estimated_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17576 entries, 0 to 17575
Data columns (total 47 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date_calc                       17576 non-null  datetime64[us]
 1   date_forecast                   17576 non-null  datetime64[us]
 2   absolute_humidity_2m:gm3        17576 non-null  float32       
 3   air_density_2m:kgm3             17576 non-null  float32       
 4   ceiling_height_agl:m            13657 non-null  float32       
 5   clear_sky_energy_1h:J           17576 non-null  float32       
 6   clear_sky_rad:W                 17576 non-null  float32       
 7   cloud_base_agl:m                15482 non-null  float32       
 8   dew_or_rime:idx                 17576 non-null  float32       
 9   dew_point_2m:K                  17576 non-null  float32       
 10  diffuse_rad:W                   17576 non-null  float32       
 11  di

In [140]:
X_train_estimated_a = X_train_estimated_a.drop(["date_calc"],axis=1)

In [141]:
def data_length_matching(train: pd.DataFrame, obs: pd.DataFrame)-> tuple[pd.DataFrame, pd.DataFrame]:
    """
    This function is intended to ensure that both the training data and
    the observed data are sorted, and contain the same number of entries. 
    """

    # Cut the data frames so that their date match.
    obs_feature_test = obs[obs['date_forecast'].isin(train['time'])].sort_values(by=['date_forecast']) # sortert etter datao
    
    # If only one of them has the date ensure that the other also has the same sorting.
    train_feature_test = train[train['time'].isin(obs['date_forecast'])].sort_values(by=['time']) # sortert etter datao

    # Would not the rest ensure this?
    print('If True same length and time stamps')
    print(len(obs_feature_test) == len(train_feature_test))
    print(len(obs_feature_test), len(train_feature_test))

    return train_feature_test, obs_feature_test

def dt64_to_float(dt64):
     year = dt64.astype('M8[Y]')
     days = (dt64 - year).astype('timedelta64[D]')
     year_next = year + np.timedelta64(1, 'Y')
     days_of_year = (year_next.astype('M8[D]') - year.astype('M8[D]')).astype('timedelta64[D]')
     dt_float = 1970 + year.astype(float) + days / (days_of_year)
     return dt_float

In [142]:
X_total = pd.concat([X_train_estimated_a, X_train_observed_a], axis = 0)
y, X = data_length_matching(train_a, X_total)

If True same length and time stamps
True
34061 34061


In [143]:
X['date_forecast'] = dt64_to_float(X['date_forecast'].to_numpy())

In [144]:
# y['time'] = dt64_to_float(y['time'].to_numpy())
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)

# Get the mask of selected features (True for selected, False for removed)
selected_features_mask = sel.get_support()

# Get the names of removed features
removed_features = [feature for feature, keep in zip(X.columns, selected_features_mask) if not keep]

# Print the names of removed features
print("Removed features:", removed_features)

Removed features: ['air_density_2m:kgm3', 'dew_or_rime:idx', 'elevation:m', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'precip_5min:mm', 'precip_type_5min:idx', 'rain_water:kgm2', 'snow_density:kgm3', 'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2', 'super_cooled_liquid_water:kgm2', 'wind_speed_w_1000hPa:ms']


In [145]:
X = X.drop(removed_features, axis=1)
X = X.fillna(0)
# y = y.drop('time', axis=1)
y = y['pv_measurement'].ravel()

In [149]:
X_new = SelectKBest(f_regression, k=5).fit_transform(X, y)
print("This is x-new")
print(X_new)

#+====================================================================================================================+
#| Additional idea: Notice that time of day parameters will probably be provided... Fitting this to a location might  |
#| give us an edge needed to fit a better algorithm for each location...                                              |
#+====================================================================================================================+


# This is something 
# Use some of scikit learn feature extraction functionality.

# VarianceThreshold

# SelectKBest

# Tree-based feature selection

# Try implementing this into: Pipeline (1.13.6.)


This is x-new
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


AttributeError: 'numpy.ndarray' object has no attribute 'get_support'

: 